<a href="https://colab.research.google.com/github/Yadav106/MyLibrary/blob/main/ISL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d prathumarikeri/indian-sign-language-isl

Dataset URL: https://www.kaggle.com/datasets/prathumarikeri/indian-sign-language-isl
License(s): CC-BY-SA-4.0
 99% 278M/281M [00:17<00:00, 22.2MB/s]
100% 281M/281M [00:17<00:00, 16.5MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/indian-sign-language-isl.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:

import os
import shutil
import random

base_folder = '/content/Indian'
train_folder = os.path.join(base_folder, 'train')
test_folder = os.path.join(base_folder, 'test')

if not os.path.exists(train_folder):
    os.makedirs(train_folder)

if not os.path.exists(test_folder):
    os.makedirs(test_folder)


characters = [str(i) for i in range(1, 10)] + [chr(i) for i in range(65, 91)]

for char in characters:
    char_folder = os.path.join(base_folder, char)
    train_char_folder = os.path.join(train_folder, char)
    test_char_folder = os.path.join(test_folder, char)

    if not os.path.exists(train_char_folder):
        os.makedirs(train_char_folder)

    if not os.path.exists(test_char_folder):
        os.makedirs(test_char_folder)

    files = os.listdir(char_folder)

    random.shuffle(files)

    train_files = files[:900]
    test_files = files[900:1200]

    for file in train_files:
        src = os.path.join(char_folder, file)
        dst = os.path.join(train_char_folder, file)
        shutil.move(src, dst)

    for file in test_files:
        src = os.path.join(char_folder, file)
        dst = os.path.join(test_char_folder, file)
        shutil.move(src, dst)

print("Data split completed!")


Data split completed!


In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
tf.__version__

'2.17.0'

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np

class_labels = [str(i) for i in range(1, 10)] + [chr(i) for i in range(65, 91)]

label_encoder = LabelEncoder()
label_encoder.fit(class_labels)

def one_hot_encode_labels(labels):
    integer_encoded = label_encoder.transform(labels)
    onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    return onehot_encoded

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1 / 255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1 / 255)

In [ ]:
training_set = train_datagen.flow_from_directory(
    "/content/Indian/train",
    target_size=(256, 256),
    batch_size=16,
    color_mode="grayscale",
    class_mode="categorical",
    classes=class_labels
)

test_set = test_datagen.flow_from_directory(
    "/content/Indian/test",
    target_size=(256, 256),
    batch_size=16,
    color_mode="grayscale",
    class_mode="categorical",
    classes=class_labels
)

Found 31500 images belonging to 35 classes.
Found 10500 images belonging to 35 classes.


In [ ]:
model = Sequential()

In [ ]:
model.add(Convolution2D(32, (3, 3), input_shape=(256, 256, 1), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(128, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(units=128, activation="relu"))
model.add(Dropout(0.5))

model.add(Dense(units=64, activation="relu"))
model.add(Dropout(0.5))

model.add(Dense(units=35, activation="softmax"))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(
    optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"]
)

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 254, 254, 32)        │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 127, 127, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 125, 125, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 60, 60, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 30, 30, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 115200)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      14,745,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 35)                  │           2,275 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,848,931 (56.64 MB)

 Trainable params: 14,848,931 (56.64 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    training_set,
    epochs=5,
    validation_data=test_set,
)




Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1969/1969 ━━━━━━━━━━━━━━━━━━━━ 233s 110ms/step - accuracy: 0.4594 - loss: 1.8681 - val_accuracy: 0.9930 - val_loss: 0.0325
Epoch 2/5
1969/1969 ━━━━━━━━━━━━━━━━━━━━ 199s 101ms/step - accuracy: 0.8716 - loss: 0.4077 - val_accuracy: 0.9951 - val_loss: 0.0150
Epoch 3/5
1969/1969 ━━━━━━━━━━━━━━━━━━━━ 201s 100ms/step - accuracy: 0.9108 - loss: 0.2718 - val_accuracy: 0.9970 - val_loss: 0.0078
Epoch 4/5
1969/1969 ━━━━━━━━━━━━━━━━━━━━ 203s 101ms/step - accuracy: 0.9317 - loss: 0.2122 - val_accuracy: 0.9960 - val_loss: 0.0256
Epoch 5/5
1969/1969 ━━━━━━━━━━━━━━━━━━━━ 198s 100ms/step - accuracy: 0.9411 - loss: 0.1882 - val_accuracy: 0.9996 - val_loss: 0.0029


In [ ]:
print("Training Accuracy:", max(history.history['accuracy']))
print("Validation Accuracy:", max(history.history['val_accuracy']))
print("Training Loss:", max(history.history['loss']))
print("Validation Loss:", max(history.history['val_loss']))

Training Accuracy: 0.9429206252098083
Validation Accuracy: 0.9996190667152405
Training Loss: 1.09959876537323
Validation Loss: 0.03245411813259125


In [ ]:
# import numpy as np

# train_predictions = np.argmax(model.predict(training_set))
# test_predictions = np.argmax(model.predict(test_set))
# train_true_labels = np.argmax(training_set.labels)
# test_true_labels = np.argmax(test_set.labels)


# train_predictions = np.expand_dims(train_predictions)
# test_predictions = np.expand_dims(test_predictions)
# train_true_labels = np.expand_dims(train_true_labels)
# test_true_labels = np.expand_dims(test_true_labels)

# train_f1 = tf.keras.metrics.F1Score()
# train_f1.update_state(train_true_labels, train_predictions)
# train_f1_result = train_f1.result().numpy()

# test_f1 = tf.keras.metrics.F1Score()
# test_f1.update_state(test_true_labels, test_predictions)
# test_f1_result = test_f1.result().numpy()

# train_recall = tf.keras.metrics.Recall()
# train_recall.update_state(train_true_labels, train_predictions)
# train_recall_result = train_recall.result().numpy()

# test_recall = tf.keras.metrics.Recall()
# test_recall.update_state(test_true_labels, test_predictions)
# test_recall_result = test_recall.result().numpy()

# print("Training F1 Score:", train_f1_result)
# print("Training Recall:", train_recall_result)

# print("Test F1 Score:", test_f1_result)
# print("Test Recall:", test_recall_result)


In [ ]:
model.save("model.h5")

In [ ]:
model.save("model.keras")

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.17.0
